# AWS NetCDF4 to Zarr Using Harmony Reformatter

**Goal**
<br/>
To direct the Harmony Zarr Reformatter to the proper subset of the MUR 1-km dataset stored in an S3 bucket within Amazon Web Services (AWS) and use it's output in conjunction with the MUR Climatology dataset (created by Mike Chin and cleaned in the notebook 'CleaningMURClimatologyData.ipynb') to create a Sea Surface Temperature (SST) anomaly dataset for use in testing runtimes on dataset loading and plotting applications. 

**Run Location**
<br/>
This notebook was run on an AWS EC2 t3.medium instance. It was discovered that for this specific analysis and dataset a t3.small instance did not provide enough memory.

**Dataset**
<br/>
MUR 1-km L4 SST netCDF4 AWS (requires AWS early access in order to view on Earthdata Search) https://podaac.jpl.nasa.gov/MEaSUREs-MUR?tab=background&sections=about%2Bdata

### Import Modules

In [14]:
%matplotlib inline
import sys
from eosdis_store import EosdisStore

import s3fs
import numpy as np
import xarray as xr
import fsspec
import zarr
import timeit
import time
import zarr
import matplotlib.pyplot as plt
from dask.distributed import Client, performance_report

import requests
import pandas as pd
from IPython.display import HTML
from json import dumps, loads
from platform import system
from netrc import netrc
from getpass import getpass
from urllib import request
from http.cookiejar import CookieJar
from os.path import join, expanduser

### Set the CMR, URS, and Harmony Endpoints
<br/>
CMR, or the Earthdata Common Metadata Repository, is a high-performance, high-quality, continuously evolving metadata system that catalogs Earth Science data and associated service metadata records. URS is the Earthdata login system, that allows (free) download access to Earthdata data. Harmony API allows you to seamlessly analyze Earth observation data from different NASA data centers.

In [2]:
cmr = "cmr.earthdata.nasa.gov"
urs = "urs.earthdata.nasa.gov"
harmony = "harmony.earthdata.nasa.gov"

cmr, urs, harmony

('cmr.earthdata.nasa.gov',
 'urs.earthdata.nasa.gov',
 'harmony.earthdata.nasa.gov')

### Earthdata Login
<br/>
An Earthdata Login account is required to access data, as well as discover restricted data, from the NASA Earthdata system. Please visit https://urs.earthdata.nasa.gov to register and manage your Earthdata Login account. This account is free to create and only takes a moment to set up.

The setup_earthdata_login_auth function will allow Python scripts to log into any Earthdata Login application programmatically. To avoid being prompted for credentials every time you run and also allow clients such as curl to log in, you can add the following to a .netrc (_netrc on Windows) file in your home directory:

    machine urs.earthdata.nasa.gov
    login <your username>
    password <your password>

Make sure that this file is only readable by the current user or you will receive an error stating "netrc access too permissive."

    $ chmod 0600 ~/.netrc

In [3]:
TOKEN_DATA = ("<token>"
              "<username>%s</username>"
              "<password>%s</password>"
              "<client_id>PODAAC CMR Client</client_id>"
              "<user_ip_address>%s</user_ip_address>"
              "</token>")


def setup_earthdata_login_auth(urs: str='urs.earthdata.nasa.gov', cmr: str='cmr.earthdata.nasa.gov'):

    # GET URS LOGIN INFO FROM NETRC OR USER PROMPTS:
    netrc_name = "_netrc" if system()=="Windows" else ".netrc"
    try:
        username, _, password = netrc(file=join(expanduser('~'), netrc_name)).authenticators(urs)
        print("# Your URS credentials were securely retrieved from your .netrc file.")
    except (FileNotFoundError, TypeError):
        print('# Please provide your Earthdata Login credentials for access.')
        print('# Your info will only be passed to %s and will not be exposed in Jupyter.' % (urs))
        username = input('Username: ')
        password = getpass('Password: ')

    # SET UP URS AUTHENTICATION FOR HTTP DOWNLOADS:
    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, urs, username, password)
    auth = request.HTTPBasicAuthHandler(manager)
    jar = CookieJar()
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener)

    # GET TOKEN TO ACCESS RESTRICTED CMR METADATA:
    ip = requests.get("https://ipinfo.io/ip").text.strip()
    r = requests.post(
        url="https://%s/legacy-services/rest/tokens" % cmr,
        data=TOKEN_DATA % (str(username), str(password), ip),
        headers={'Content-Type': 'application/xml', 'Accept': 'application/json'}
    )
    return r.json()['token']['id']


# Provide URS credentials for HTTP download auth & CMR token retrieval:
_token = setup_earthdata_login_auth(urs=urs, cmr=cmr)

# Your URS credentials were securely retrieved from your .netrc file.


### Obtain Dataset Metadata

In [4]:
SHORTNAME = "MUR-JPL-L4-GLOB-v4.1"

In [5]:
r = requests.get(url=f"https://{cmr}/search/collections.umm_json", 
                 params={'provider': "POCLOUD", 
                         'ShortName': SHORTNAME, 
                         'token': _token})

mur_coll = r.json()
mur_coll_meta = mur_coll['items'][0]['meta']
mur_coll_meta

{'revision-id': 29,
 'deleted': False,
 'format': 'application/vnd.nasa.cmr.umm+json',
 'provider-id': 'POCLOUD',
 'user-id': 'mgangl',
 'has-formats': False,
 'associations': {'variables': ['V2028632042-POCLOUD',
   'V2028632044-POCLOUD',
   'V2028632047-POCLOUD',
   'V2028668049-POCLOUD']},
 's3-links': ['podaac-ops-cumulus-public/MUR-JPL-L4-GLOB-v4.1/',
  'podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/'],
 'has-spatial-subsetting': False,
 'native-id': 'GHRSST+Level+4+MUR+Global+Foundation+Sea+Surface+Temperature+Analysis+(v4.1)',
 'has-transforms': False,
 'has-variables': True,
 'concept-id': 'C1996881146-POCLOUD',
 'revision-date': '2021-07-26T17:41:06.284Z',
 'granule-count': 0,
 'has-temporal-subsetting': False,
 'concept-type': 'collection'}

## Setup for Regional Tests

### Period and Region of Interest

In [9]:
start_date = "2019-08-01"
end_date = "2019-08-01"

minlat = 18
maxlat = 23
minlon = -160
maxlon = -154

### Request to Harmony API: Zarr Reformatter
<br/>
But we'll use the Harmony API's Zarr Reformatter service instead of downloading the entire granule. The zarr format will allow us to open and download/read just the data that we require for our Amazon Basin study area.
<br/>
<br/>
If you have a jobID you'd like to re-visit instead of running this command again, modify the cell below to set the async_jobId then skip the one immediately after. You can continue from 'Query for the job status and links'.
<br/>
<br/>
If you are running for the first time, proceed to the next cells to submit the harmony request.

In [7]:
# async_jobId = "74632965-e9fd-407d-aa78-c2b4306eeda6"  # jobId belongs to dev. You wont have access.
async_jobId = None

See important usage note below if this is your first time submitting a request to the Zarr Reformatter service.

The Zarr Reformatter service operates on an input Collection concept-id (a CMR construct). The service will accept more user-friendly inputs (like a Collection ShortName) in future releases. Here's how you identify the CMR concept-id for the JPL MUR 1-km SST dataset:

In [8]:
collection_concept_id = mur_coll_meta['concept-id']
collection_concept_id

'C1996881146-POCLOUD'

In [10]:
lat = '(' + str(minlat) + ':' + str(maxlat) + ')'
lon = '(' + str(minlon) + ':' + str(maxlon) + ')' 
time = '(' + "\"" + str(start_date) + 'T09:00:00Z' + "\"" + ':\"' + str(end_date) + 'T09:00:00Z' + "\"" + ')'

In [11]:
async_url = f'https://{harmony}/{collection_concept_id}/ogc-api-coverages/1.0.0/collections/all/coverage/rangeset?subset=lat{lat}&subset=lon{lon}&subset=time{time}&format=application/x-zarr'
if async_jobId is None:
    print('Request URL: ', async_url)
    async_response = request.urlopen(async_url)
    async_results = async_response.read()
    async_json = loads(async_results)
    print(dumps(async_json, indent=2))
    async_jobId = async_json['jobID']

Request URL:  https://harmony.earthdata.nasa.gov/C1996881146-POCLOUD/ogc-api-coverages/1.0.0/collections/all/coverage/rangeset?subset=lat(18:23)&subset=lon(-160:-154)&subset=time("2019-08-01T09:00:00Z":"2019-08-01T09:00:00Z")&format=application/x-zarr
{
  "username": "matthew.a.thompson",
  "status": "running",
  "message": "The job is being processed",
  "progress": 0,
  "createdAt": "2021-08-16T17:47:30.360Z",
  "updatedAt": "2021-08-16T17:47:30.360Z",
  "links": [
    {
      "href": "https://harmony.earthdata.nasa.gov/jobs/05a2ebd6-0307-48c3-90fc-d4c56e412057?page=1&limit=2000",
      "title": "The current page",
      "type": "application/json",
      "rel": "self"
    }
  ],
  "request": "https://harmony.earthdata.nasa.gov/C1996881146-POCLOUD/ogc-api-coverages/1.0.0/collections/all/coverage/rangeset?subset=lat(18%3A23)&subset=lon(-160%3A-154)&subset=time(%222019-08-01T09%3A00%3A00Z%22%3A%222019-08-01T09%3A00%3A00Z%22)&format=application%2Fx-zarr",
  "numInputGranules": 1,
  "jobI

Format and display the complete url to the Harmony API job:

In [12]:
job_url = f'https://{harmony}/jobs/{async_jobId}'
job_url

'https://harmony.earthdata.nasa.gov/jobs/05a2ebd6-0307-48c3-90fc-d4c56e412057'

Query for the job status and links in case the request is still processing:

In [15]:
start_time = timeit.default_timer()

while True:
    loop_response = request.urlopen(job_url)
    loop_results = loop_response.read()
    job_json = loads(loop_results)
    if job_json['status'] != 'running':
        break
    print(f"# Job status is running. Progress is {job_json['progress']}. Trying again.")
    time.sleep(5)

links = []
if job_json['status'] == 'successful' and job_json['progress'] == 100:
    print("# Job progress is 100%. Links to job outputs are displayed below:")
    links = [link['href'] for link in job_json['links']]
    display(links)
    
elapsed = timeit.default_timer() - start_time
print(elapsed)

# Job status is running. Progress is 0. Trying again.
# Job status is running. Progress is 0. Trying again.
# Job status is running. Progress is 0. Trying again.
# Job status is running. Progress is 0. Trying again.
# Job status is running. Progress is 0. Trying again.
# Job status is running. Progress is 0. Trying again.
# Job status is running. Progress is 0. Trying again.
# Job status is running. Progress is 0. Trying again.
# Job status is running. Progress is 0. Trying again.
# Job status is running. Progress is 0. Trying again.
# Job status is running. Progress is 0. Trying again.
# Job status is running. Progress is 0. Trying again.
# Job status is running. Progress is 0. Trying again.
# Job status is running. Progress is 0. Trying again.
# Job status is running. Progress is 0. Trying again.
# Job status is running. Progress is 0. Trying again.
# Job status is running. Progress is 0. Trying again.
# Job status is running. Progress is 0. Trying again.
# Job status is running. Pro

['https://harmony.earthdata.nasa.gov/stac/05a2ebd6-0307-48c3-90fc-d4c56e412057/',
 'https://harmony.earthdata.nasa.gov/cloud-access.sh',
 'https://harmony.earthdata.nasa.gov/cloud-access',
 's3://harmony-prod-staging/public/harmony/netcdf-to-zarr/f228b144-a4a7-405c-8a21-785b7df7c0b4/',
 's3://harmony-prod-staging/public/harmony/netcdf-to-zarr/f228b144-a4a7-405c-8a21-785b7df7c0b4/20190801090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.zarr',
 'https://harmony.earthdata.nasa.gov/jobs/05a2ebd6-0307-48c3-90fc-d4c56e412057?page=1&limit=2000']

149.523580299


### Access URL for the Output Zarr File
<br/>
The new zarr dataset is staged for us in an S3 bucket. The url is the last one in the list shown above.

Select the url and display below:

In [13]:
links

['https://harmony.earthdata.nasa.gov/stac/74632965-e9fd-407d-aa78-c2b4306eeda6/',
 'https://harmony.earthdata.nasa.gov/cloud-access.sh',
 'https://harmony.earthdata.nasa.gov/cloud-access',
 's3://harmony-prod-staging/public/harmony/netcdf-to-zarr/2f0c87d2-20d1-4054-a11c-9803c5810b91/',
 's3://harmony-prod-staging/public/harmony/netcdf-to-zarr/2f0c87d2-20d1-4054-a11c-9803c5810b91/20190801090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.zarr',
 's3://harmony-prod-staging/public/harmony/netcdf-to-zarr/2f0c87d2-20d1-4054-a11c-9803c5810b91/20190802090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.zarr',
 's3://harmony-prod-staging/public/harmony/netcdf-to-zarr/2f0c87d2-20d1-4054-a11c-9803c5810b91/20190803090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.zarr',
 's3://harmony-prod-staging/public/harmony/netcdf-to-zarr/2f0c87d2-20d1-4054-a11c-9803c5810b91/20190804090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.zarr',
 's3://harmony-prod-staging/public/harmony/netcdf-to-zarr/2f0c87d2-20d1-4

In [14]:
links[4]

's3://harmony-prod-staging/public/harmony/netcdf-to-zarr/2f0c87d2-20d1-4054-a11c-9803c5810b91/20190801090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.zarr'

In [15]:
links[176]

's3://harmony-prod-staging/public/harmony/netcdf-to-zarr/2f0c87d2-20d1-4054-a11c-9803c5810b91/20200120090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.zarr'

In [16]:
zarr_urls = links[4:177]
zarr_urls

['s3://harmony-prod-staging/public/harmony/netcdf-to-zarr/2f0c87d2-20d1-4054-a11c-9803c5810b91/20190801090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.zarr',
 's3://harmony-prod-staging/public/harmony/netcdf-to-zarr/2f0c87d2-20d1-4054-a11c-9803c5810b91/20190802090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.zarr',
 's3://harmony-prod-staging/public/harmony/netcdf-to-zarr/2f0c87d2-20d1-4054-a11c-9803c5810b91/20190803090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.zarr',
 's3://harmony-prod-staging/public/harmony/netcdf-to-zarr/2f0c87d2-20d1-4054-a11c-9803c5810b91/20190804090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.zarr',
 's3://harmony-prod-staging/public/harmony/netcdf-to-zarr/2f0c87d2-20d1-4054-a11c-9803c5810b91/20190805090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.zarr',
 's3://harmony-prod-staging/public/harmony/netcdf-to-zarr/2f0c87d2-20d1-4054-a11c-9803c5810b91/20190806090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.zarr',
 's3://harmony-prod-staging/public/harmo

### Access Credentials for the Output Zarr File
<br/>
Credentials provided at the third and fourth urls in the list grant authenticated access to your staged S3 resources.

Grab the credentials as a JSON string, load to a Python dictionary, and display their expiration date:

In [19]:
with request.urlopen(f"https://{harmony}/cloud-access") as f:
    creds = loads(f.read())

creds['Expiration']

'2021-08-17T01:54:43.000Z'

In [20]:
zarr_fs = s3fs.S3FileSystem(
    key=creds['AccessKeyId'],
    secret=creds['SecretAccessKey'],
    token=creds['SessionToken'],
    client_kwargs={'region_name':'us-west-2'},
)

In [31]:
mur_test = xr.open_dataset(zarr_fs.get_mapper(root=links[4], check=False), engine='zarr')
mur_test.nbytes

28510632004

### Open Staged Zarr File with s3fs

In [20]:
start_time = timeit.default_timer()

variables = [
    'analysed_sst',
    'mask'
]

# def subset(ds):
#     subset_ds = ds[variables].sel(
#         lat=slice(minlat, maxlat),   # Removing the variables selection uncovers accurate error message
#         lon=slice(minlon, maxlon)
#     )
#     return subset_ds

def subset(ds):
    subset_ds = ds.sel(
        lat=slice(minlat, maxlat),
        lon=slice(minlon, maxlon)
    )
    return subset_ds

mur_hawaii = xr.open_mfdataset(
    paths=[zarr_fs.get_mapper(root=f, check=False) for f in zarr_urls],
    preprocess=subset,
#     combine='by_coords',
    consolidated=False,
#     mask_and_scale=True,
#     decode_cf=True,
#     cache=False,
#     parallel=True,
    engine='zarr'
).chunk({"time": 30, "lat": 100, "lon": 100})

elapsed = timeit.default_timer() - start_time
print(elapsed)

ValueError: Every dimension needs a coordinate for inferring concatenation order

In [ ]:
print(mur_hawaii.tree())

In [ ]:
print(mur_hawaii.analysed_sst.info)

### Add in NAN Values for Land to MUR Data
<br/>
We use the mask dimension to replace temperature values from land observations with NaN so that they are not factored in to our calculations. The mask variable has a value for each coordinate pair representing which surface the temperature was collected from (land, open-sea, ice, etc.).

In [ ]:
mur_hawaii_sst = mur_hawaii['analysed_sst'].where(mur_hawaii.mask == 1)

### Convert Temperatures to Celsius
<br/>
The dataset is stored with temperatures measured in Kelvin. This converts it to Celsius for ease of understanding and analysis.

In [ ]:
mur_hawaii_sst = mur_hawaii_sst - 273.15

In [ ]:
mur_hawaii_sst

### Open MUR Climatology for Hawaii

In [ ]:
mur_clim = xr.open_dataarray(
    "../data/MURClimatology.nc",
    chunks={"time": 30, "lat": 100, "lon": 100}
)

In [ ]:
mur_clim

### Drop the Leap Day

In [ ]:
mur_clim = mur_clim.where(mur_clim["time"] != np.datetime64('2004-02-29T09:00:00', 'ns'), drop=True)

### Create Subset Dataset

In [ ]:
mur_clim_jan = mur_clim[0:20]

In [ ]:
mur_clim_subset = mur_clim[212:]

In [ ]:
mur_clim_subset = xr.concat([mur_clim_subset, mur_clim_jan], dim="time")

In [ ]:
mur_clim_subset = mur_clim_subset.assign_coords({"time": mur_hawaii_sst["time"]})

In [ ]:
mur_clim_subset

### Create SST Anomaly Dataset

In [ ]:
sst_anomaly = mur_hawaii_sst - mur_clim_subset

In [ ]:
sst_anomaly

### Find Daily Average SST Anomaly for Time Series

In [ ]:
sst_anomaly_mean_ts = sst_anomaly.mean(['lat', 'lon'])

In [ ]:
sst_anomaly_mean_ts

### Find Average SST Anomaly for Each Coordinate Pair for Spatial Plot

In [ ]:
sst_anomaly_mean_sp = sst_anomaly.mean(['time'])

In [ ]:
sst_anomaly_mean_sp

## Regional Tests

### Regional SST Anomaly Averaged Time Series, August 1st, 2019 - January 20th, 2020

In [ ]:
start_time = timeit.default_timer()

sst_anomaly_mean_ts.plot()

elapsed = timeit.default_timer() - start_time
print(elapsed)

### Regional SST Anomaly Averaged Spatial Plot, August 1st, 2019 - January 20th, 2020

In [ ]:
start_time = timeit.default_timer()

sst_anomaly_mean_sp.plot()

elapsed = timeit.default_timer() - start_time
print(elapsed)